<a href="https://colab.research.google.com/github/RickyDoan/DL-Tensor-Flow-Prediction/blob/main/TF_CNN_Flowers_Transferlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
import PIL
import os
import cv2
import pathlib
import tensorflow_hub as hub
import tf_keras as tfk
import PIL.Image as Image

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
# ! pip install tensorflow tensorflow-hub


In [ ]:
data_dir = '/content/drive/MyDrive/A Learning Tensor Flow/CNN-Image Classification/datasets/flower_photos'

In [ ]:
import pathlib

data_dir = pathlib.Path(data_dir)
data_dir


In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
roses = list(data_dir.glob('roses/*'))
roses[:5]

In [ ]:
PIL.Image.open(str(roses[1]))

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

In [ ]:
class_name = train_ds.class_names
print(class_name)

In [ ]:
plt.figure(figsize=(10,10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3,3,i+1)
    plt.imshow(images[i].numpy().astype('uint8'))
    plt.title(class_name[labels[i]])
    plt.axis('off')
plt.show()

In [ ]:
tulips = list(data_dir.glob('tulips/*'))
plt.figure(figsize=(10,7))
for i in range(9):
    ax = plt.subplot(3,3,i+1)
    plt.imshow(Image.open(str(tulips[i])))
    plt.axis('off')
plt.show()

In [ ]:
len(class_name)

In [ ]:
mobilenet_v2 ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
inception_v3 = "https://tfhub.dev/google/imagenet/inception_v3/classification/5"

In [ ]:
IMAGE_SHAPE = (224,224)
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = tfk.Sequential([
  hub.KerasLayer(feature_extractor_model, input_shape=IMAGE_SHAPE+(3,), trainable=False)
])
num_of_flowers = 5

model = tfk.Sequential([
  tfk.layers.Rescaling(1./255, input_shape=IMAGE_SHAPE+(3,)),
  pretrained_model_without_top_layer,
  tfk.layers.Dense(num_of_flowers)
])

model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=tfk.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

with tf.device('/device:GPU:0'):
      history = model.fit(train_ds, validation_data=val_ds, epochs=10)
model.evaluate(val_ds)

In [ ]:
IMG_SHAPE = (224,224)
pretrained_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
pretrained_tf = tfk.Sequential([
  hub.KerasLayer(pretrained_model, input_shape=IMAGE_SHAPE+(3,), trainable=False)
])

data_augumentation = tfk.Sequential([
  tfk.layers.RandomFlip('horizontal', input_shape=IMG_SHAPE+(3,)),
  tfk.layers.RandomRotation(0.2),
  tfk.layers.RandomZoom(0.1)
])

model_aug = tfk.Sequential([
  tfk.layers.Rescaling(1./255, input_shape=IMG_SHAPE+(3,)),
  pretrained_tf,
  tfk.layers.Dropout(0.2),
  tfk.layers.Dense(5)
])

model_aug.summary()

In [ ]:
# model_aug.compile(
#     optimizer='adam',
#     loss=tfk.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=['accuracy']
# )
# with tf.device('/device:GPU:0'):
#       history = model_aug.fit(train_ds, validation_data=val_ds, epochs=10)
# model_aug.evaluate(val_ds)

In [ ]:
for image_batch, label_batch in val_ds:
  print(image_batch.shape)
  print(label_batch.shape)
  break

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred_all = []  # To store all predictions
y_true_all = []  # To store all true labels

for image_batch, label_batch in val_ds:
  y_pred = model.predict(image_batch)
  y_pred = np.argmax(y_pred, axis=1)

  y_pred_all.extend(y_pred)  # Accumulate predictions
  y_true_all.extend(label_batch.numpy())  # Accumulate true labels

print(classification_report(y_true_all, y_pred_all, target_names=class_names))
